<a href="https://colab.research.google.com/github/jonnes-nascimento/machine_learning/blob/master/Introdu%C3%A7%C3%A3o_a_Machine_Learning_e_Classifica%C3%A7%C3%A3o_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando e tratando os dados

In [1]:
import pandas as pd
from datetime import datetime

# importando a base de dados
uri = 'https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv'
dados = pd.read_csv(uri)

# tratamento da base de dados 
#   renomeando as colunas
a_renomear = {
    'mileage_per_year' : 'milhas_por_ano',
    'model_year' : 'ano_do_modelo',
    'price' : 'preco',
    'sold' : 'vendido'
}
dados = dados.rename(columns=a_renomear)

#   alterando os valores 'yes' ou 'no' para o padrão binário
a_trocar = {
    'yes' : 1,
    'no' : 0
}
dados.vendido = dados.vendido.map(a_trocar)

#   criacao de novas colunas
dados['km_por_ano'] = dados.milhas_por_ano * 1.60934

ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo

#   apagando as colunas desnecessarias
dados = dados.drop(columns = ['Unnamed: 0', 'milhas_por_ano', 'ano_do_modelo'], axis = 'columns')

dados.head()

,preco,vendido,km_por_ano,idade_do_modelo
0,30941.02,1,35085.22134,20
1,40557.96,1,12622.05362,22
2,89627.50,0,11440.79806,14
3,95276.14,0,43167.32682,5
4,117384.68,1,12770.11290,6


# Analisando o modelo

In [2]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = dados[['km_por_ano', 'idade_do_modelo', 'preco']]
y = dados['vendido']

SEED = 5
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

# calculando as previsoes
previsoes = modelo.predict(teste_x)

# calculo da acuracia
acuracia = accuracy_score(teste_y, previsoes) * 100

print("A acuracia foi de %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acuracia foi de 58.00%


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# Analisando com o uso de dummy classifiers

In [3]:
from sklearn.dummy import DummyClassifier

dummy_stratified  = DummyClassifier()
dummy_stratified .fit(treino_x, treino_y)

previsoes = dummy_stratified .predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print("A acurácia do dummy_stratified  foi de %.2f%%" % acuracia)

A acurácia do dummy_stratified  foi de 52.44%


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [4]:
from sklearn.dummy import DummyClassifier

dummy_mostfrequent  = DummyClassifier(strategy='most_frequent')
dummy_mostfrequent .fit(treino_x, treino_y)

previsoes = dummy_mostfrequent .predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print("A acurácia do dummy_mostfrequent  foi de %.2f%%" % acuracia)

A acurácia do dummy_mostfrequent  foi de 58.00%


# Testando o modelo SVC ao invés do LinearSVC


In [6]:
# analisando o código acima, verificamos que a estratégia 'most_frequent' foi a de melhor resultado. A usaremos como baseline para comparacao

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

SEED = 5
np.random.seed(SEED)

# como usaremos o SVC, devemos reescalar o eixo X para se adaptar ao eixo Y, por isso, as variaveis de teste e treino em X serao renomeadas
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y) # como o escalador tomara como base o eixo y, teremos novos valores de x

print("Treinaremos com %d elementos e testaremos %d elementos" % (len(treino_x), len(teste_x)))

# transforma a escala do eixo X para se adequar melhor ao modelo SVC
# o eixo Y não se altera pois já está em uma escala condizente (ampla)
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print("A acurácia foi de %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos 2500 elementos
A acurácia foi de 77.48%


In [0]:
# verifica-se acima que a acurácia do SVC é muito melhor do que a baseline que definimos!